In [1]:
import pandas as pd

In [2]:
path='/Users/juanignaciosolerno/Downloads/julius/datasets/title_discovery_no_parsed_jt (3).csv'
df = pd.read_csv(path)
df.sample(30)


,doc_id,doc_title,doc_company,doc_location,parsed_jt,source,Job Title Present
2882,indeed_job_02368847752d54b8,Aliana Produce - Perishables Rep - Part-Time,HEB,"Richmond, TX 77407",NaN,indeed,Yes
13649,4566871,Deliver with Uber Eats,Uber,"Fredericksburg, VA",NaN,linkedin,No
4255,bb9d104cf0bee4a4,Amazon careers - Full-Time / Benefits Day One ...,Amazon,"Knox, NY 12107",NaN,indeed,No
8626,1605032,Deliver with Uber Eats,Uber,"Webster, TX",NaN,linkedin,No
14615,5347591,Deliver with Uber Eats,Uber,"Richmond, VA",NaN,linkedin,No
6053,7415639,Deliver with Uber Eats,Uber,"Lavaca, AR",NaN,linkedin,No
15163,7621612,Deliver with Uber Eats,Uber,"Lake Colorado City, TX",NaN,linkedin,No
4248,e6f35dcd32e8cce6,Corpus 12 eStores - eStore Curbie - Part-Time,HEB,"Corpus Christi, TX",NaN,indeed,Yes
7587,6403150,Deliver with Uber Eats,Uber,"Haysville, KS",NaN,linkedin,No
4813,bff48d1b2829a4f7,Bolt Down - 2nd Shift,ABB,"Fort Smith, AR",NaN,indeed,No


In [3]:
df.shape

(16826, 7)

In [4]:
titles_to_parse = list(set(df[df['Job Title Present']=='Yes']['doc_title']))

In [5]:
titles_to_parse[:10]

['LaGrange Market - Perishables Rep - Part-Time',
 'Lakeway Service - Service Lead - Part-Time',
 'Ennis Deli - Food Service Rep - Part-Time',
 'VP, Sales',
 '2024 Summer Intern - BrightDrop: Strategy',
 'Summerwood Seafood - Perishables Rep - Part-Time',
 'Houston 67 Market - Perishables Rep - Part-Time',
 'Brenham Service - Service Rep - Part-Time',
 'Lead - 2nd Shift (M-F 1:30pm-10pm)',
 'Summer 2024 Intern Program: Engineering Roles (Massachusetts)']

In [6]:
len(titles_to_parse)

1289

In [7]:
from openai import OpenAI
openai_client = OpenAI()

In [8]:
from pydantic import BaseModel
class JobTitleExtraction(BaseModel):
    job_title: list[str]
    experience_level: list[str]
    specialization: list[str]
    location: list[str]

In [11]:
def llm_ner_job_title(job_title, client):
    if not client:
        print('No client for LLM usage')
        return None
    
    def prepare_job_title(job_title):
        if not isinstance(job_title, str):
            job_title = str(job_title)
        delimiters = ['-', '/', '_', ',', '.']
        for delimiter in delimiters:
            job_title = job_title.replace(delimiter, f' {delimiter} ')  # Separate special characters to facilitate labelling
        return job_title


    # Separe the special characters from the alphabetic characters
    job_title_prepared = prepare_job_title(job_title)
    #print(job_title_prepared)


    #chat_completion = client.chat.completions.create(
    chat_completion = client.beta.chat.completions.parse(
        model='gpt-4o-mini',
        temperature=0.0,
        seed=42,
        messages=[
            {
            "role": "system",
            "content": """
            You are an expert in Renewable Energies with specialized skills in performing Named Entity Recognition (NER) on job titles. Your task is to extract specific entities from the provided job titles, without altering the original text.

            The entities to extract are:
            - 'job_title': Words directly related to the core occupation (example: 'Wind Turbine Technician')
            - 'experience_level': Words indicating the worker's seniority or experience level (example: 'Senior', 'III'). 
            - 'location': Any words that refer to geographical locations (example: 'California')
            - 'specialization': Additional words that give specific context or specialization to the job title (example: 'offshore wind farms')

            Guidelines:
            - Your output must be a JSON object, where each key is an entity, and the values are lists of the words forming each entity.
            - Do not modify the original job title. Only include words present in the input.
            - Be careful to avoid problematic characters in the output.

            IMPORTANT
            Things or entities you shoud not include:
            - salary ('USD 1000')
            - shift, working hours or schedule ('From 8:00 to 12:00')
            - employment type ('Part - Time')
            - program type ("Summer College" in "Summer College Intern")

            Examples:
            - For the job title 'Financial Analyst II, emergent markets', return: {"job_title": ["Financial", "Analyst"], "experience_level": ["II"], "specialization": ["emergent", "markets"]}
            - For 'RS Avionics Electrical Technician Lead', return: {"job_title": ["Electrical", "Technician"], "experience_level": ["Lead"], "specialization": ["RS", "Avionics"]}
            - For 'Principal Renewables Energy Land Planner (Mid-Senior Level)', return: {"job_title": ["Land", "Planner"], "experience_level": ["Principal"], "specialization": ["Renewables", "Energy"]}

            IMPORTANT:
            - Perform NER only on the job title itself.
            - Do not include any explanations or additional text in the output—just the extracted entities in JSON format.
            """
            },
            {
                "role": "user",
                "content": f"Recognize the specified named entities on this job title: {job_title_prepared}"
            }
        ],
        response_format=JobTitleExtraction,


    )

    response_content = chat_completion.choices[0].message.content
    response_json = eval(response_content)

    return response_json

In [13]:
test  = 'Supv,Customer Experience-Billing'
response = llm_ner_job_title(job_title=test, client=openai_client)
response

{'job_title': ['Customer', 'Experience'],
 'experience_level': ['Supv'],
 'specialization': ['Billing'],
 'location': []}

In [14]:
def generate_iob_tags(job_title, ner_output):
    """
    Generate IOB tags for the given job title based on NER output.

    :param job_title: str, the raw job title string
    :param ner_output: dict, the NER output with keys 'job_title', 'experience_level', 'specialization'
    :return: list, IOB tags corresponding to the job title words
    """

    def prepare_job_title(job_title):
        if not isinstance(job_title, str):
            job_title = str(job_title)
        delimiters = ['-', '/', '_', ',', '.']
        for delimiter in delimiters:
            job_title = job_title.replace(delimiter, f' {delimiter} ')  # Separate special characters to facilitate labelling
        return job_title
    
    # Separe the special characters from the alphabetic characters
    job_title_prepared = prepare_job_title(job_title)
    
    # Split the job title into words
    job_title_tokens = job_title_prepared.split()

    # Create a list to store IOB tags
    iob_tags = ['O'] * len(job_title_tokens)

    # Helper function to mark the IOB tags
    def mark_iob(tags, tokens, label):
        for idx, token in enumerate(tokens):
            token_index = job_title_tokens.index(token)
            if idx == 0:
                tags[token_index] = f'B-{label}'  # Beginning tag
            else:
                tags[token_index] = f'I-{label}'  # Inside tag

    # Process each entity and assign corresponding IOB tags
    if 'experience_level' in ner_output:
        mark_iob(iob_tags, ner_output['experience_level'], 'exp')
    
    if 'job_title' in ner_output:
        mark_iob(iob_tags, ner_output['job_title'], 'job')
    
    if 'specialization' in ner_output:
        mark_iob(iob_tags, ner_output['specialization'], 'spe')
    
    if 'location' in ner_output:
        mark_iob(iob_tags, ner_output['location'], 'loc')

    return (job_title_prepared,iob_tags)

In [15]:
# Example usage
job_title = "Supv, Customer Experience-Billing"


ner_output = {
    "job_title": ["Customer", "Experience"],
    "experience_level": ["Supv"],
    "specialization": ["Billing"]
}

training_tuple = generate_iob_tags(job_title, ner_output)
training_tuple

('Supv ,  Customer Experience - Billing',
 ['B-exp', 'O', 'B-job', 'I-job', 'O', 'B-spe'])

In [21]:
training_tuples = []
for job_title in titles_to_parse[10:20]:
    
    
    ner_output = llm_ner_job_title(job_title,client=openai_client)

    training_tuple = generate_iob_tags(job_title, ner_output)
    
    training_tuples.append(training_tuple)
    

In [22]:
training_tuples

[('VP ,  Federal Affairs and Head of Washington Office',
  ['B-job', 'O', 'I-job', 'I-job', 'O', 'I-job', 'O', 'B-loc', 'I-job']),
 ('Bulverde Produce  -  Perishables Rep  -  Part - Time',
  ['B-loc', 'O', 'O', 'B-job', 'I-job', 'O', 'O', 'O', 'O']),
 ('Courier / DOT - 3', ['B-job', 'O', 'B-spe', 'O', 'B-exp']),
 ('Pearsall Deli  -  Food Service Rep  -  Part - Time',
  ['B-loc', 'O', 'O', 'B-job', 'I-job', 'I-job', 'O', 'O', 'O', 'O']),
 ('Cleburne Market  -  Perishables Rep  -  Part - Time',
  ['B-loc', 'O', 'O', 'B-job', 'I-job', 'O', 'O', 'O', 'O']),
 ('Austin 14 Market  -  Perishables Rep  -  Part - Time',
  ['B-loc', 'I-loc', 'I-loc', 'O', 'B-job', 'I-job', 'O', 'O', 'O', 'O']),
 ('Corpus 18 CFT  -  Cross Functional Rep  -  Part - Time',
  ['B-loc',
   'I-loc',
   'I-loc',
   'O',
   'B-job',
   'I-job',
   'I-job',
   'O',
   'O',
   'O',
   'O']),
 ('Plano 01 Market  -  Perishables Rep  -  Part - Time',
  ['B-loc', 'O', 'I-loc', 'O', 'B-job', 'I-job', 'O', 'O', 'O', 'O']),
 ('Cl

In [23]:
training_dataset = pd.DataFrame(columns=['job_title_splitted','iob_tags'],data=training_tuples)
training_dataset

,job_title_splitted,iob_tags
0,"VP , Federal Affairs and Head of Washington O...","[B-job, O, I-job, I-job, O, I-job, O, B-loc, I..."
1,Bulverde Produce - Perishables Rep - Part ...,"[B-loc, O, O, B-job, I-job, O, O, O, O]"
2,Courier / DOT - 3,"[B-job, O, B-spe, O, B-exp]"
3,Pearsall Deli - Food Service Rep - Part - ...,"[B-loc, O, O, B-job, I-job, I-job, O, O, O, O]"
4,Cleburne Market - Perishables Rep - Part -...,"[B-loc, O, O, B-job, I-job, O, O, O, O]"
5,Austin 14 Market - Perishables Rep - Part ...,"[B-loc, I-loc, I-loc, O, B-job, I-job, O, O, O..."
6,Corpus 18 CFT - Cross Functional Rep - Par...,"[B-loc, I-loc, I-loc, O, B-job, I-job, I-job, ..."
7,Plano 01 Market - Perishables Rep - Part -...,"[B-loc, O, I-loc, O, B-job, I-job, O, O, O, O]"
8,Class B Driver HOT Delivery Home Daily,"[B-exp, I-exp, B-job, B-spe, I-spe, I-spe, I-spe]"
9,"Summer 2024 Intern Program: Buffalo , NY Roles","[O, O, B-exp, O, B-loc, O, I-loc, O]"
